In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]
class Cascade_info():
    def __init__(self):
        self.view = 0
        self.click = 0
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [4]:
q_ids = all_groups['QueryId'].unique()

In [5]:
q_docs_act = dict()

In [6]:
for qid in tqdm(q_ids):
    q_docs_act[qid] = set()
    tmp = all_groups[all_groups.QueryId==qid]
    tmp_docs = tmp['DocumentId'].unique()
    for doc in tmp_docs:
        q_docs_act[qid].add(doc_to_host[transform(doc, ill_url_ids)])

100%|██████████| 6311/6311 [00:09<00:00, 655.85it/s]


In [7]:
with open("q_docs_act.pickle", 'wb') as file:
    pickle.dump(q_docs_act, file)

In [8]:
double_q_dict = dict()

In [9]:
for key in tqdm(q_docs_act):
    double_q_dict[key] = dict()
    for elem in q_docs_act[key]:
        double_q_dict[key][elem]=Cascade_info()

100%|██████████| 6311/6311 [00:01<00:00, 5100.43it/s]


In [10]:
exists = set(os.listdir('host_query'))

In [11]:
for key1 in tqdm(double_q_dict):
    if 'qh_{}.pickle'.format(key1) in exists:
        with open('host_query/qh_{}.pickle'.format(key1), 'rb') as file:
            obj = pickle.load(file)
            #корректный каскад
            for session in obj:
                clicked_set = set()
                for num in session[3]:
                    #if num>-1:
                    clicked_set.add(num)
                if len(clicked_set)>0:
                    for num in session[2]:
                        if num not in clicked_set and num in q_docs_act[key1]:
                            double_q_dict[key1][num].view+=1
                        elif num in clicked_set:
                            if num in q_docs_act[key1]:
                                double_q_dict[key1][num].view+=1
                                double_q_dict[key1][num].click+=1
                            break

100%|██████████| 6311/6311 [00:15<00:00, 416.07it/s]


In [12]:
with open("QHP_casc.pickle", 'wb') as file:
    pickle.dump(double_q_dict, file)

In [13]:
1+5

6